# Portfolio Analysis for ROI CatBoost portfolio

## Set up data set

In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../lending_club') # path the the directory
import config
from model_prep import divide_by_term, split_data

In [2]:
accepted_loans = pd.read_parquet(config.DATAPATH / 'approved.parquet', engine='fastparquet')
accepted_60 = divide_by_term(accepted_loans, term_length=60)
accepted_36 = divide_by_term(accepted_loans, term_length=36)
accepted_60 = accepted_60.loc[:, config.SELECTED_FEATURES + [config.TARGET_COL]]
accepted_36 = accepted_36.loc[:, config.SELECTED_FEATURES + [config.TARGET_COL]]

## CatBoost Modeling 

In [3]:
from catboost import CatBoostClassifier

In [4]:
best_60 = CatBoostClassifier().load_model('CatBoost60.model')
best_36 = CatBoostClassifier().load_model('CatBoost36.model')

## ROI on the Entire Portfolio

In [5]:
accepted_60['prediction'] = best_60.predict(accepted_60.drop(['addr_state',
                                                              'emp_title',
                                                               config.TARGET_COL], axis=1))
accepted_36['prediction'] = best_36.predict(accepted_36.drop(['addr_state',
                                                              'emp_title',
                                                               config.TARGET_COL], axis=1))
combined = pd.concat([accepted_60,accepted_36],axis=0)

In [6]:
portfolio_df = combined.loc[combined['prediction']==1,:]
PnL_series = accepted_loans.PnL
portfolio_df = portfolio_df.merge(PnL_series, how='inner', left_index=True, right_index=True)
print(portfolio_df.PnL.sum()/portfolio_df.loan_amnt.sum())

0.12441867617314067


In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(combined['loan_status'], combined['prediction'])
cm_df = pd.DataFrame(cm, columns=['Pred: Charged Off', 'Pred: Fully Paid'], index=['True: Charged Off', 'True: Fully Paid'])
cm_df

,Pred: Charged Off,Pred: Fully Paid
True: Charged Off,63911,31583
True: Fully Paid,157372,408037


In [13]:
combined = combined.merge(PnL_series, how='inner', left_index=True, right_index=True)
combined.query('PnL>0')['PnL'].sum()/combined.query('PnL>0')['loan_amnt'].sum()

0.1690489370222356

In [14]:
combined.query('loan_status==1')['PnL'].sum()/combined.query('loan_status==1')['loan_amnt'].sum()

0.16889954412219232

In [19]:
def calculate_ROI(df, filterCol, filterValues):
    df2 = df.loc[~df[filterCol].isin(filterValues),:]
    return (df2, df2['PnL'].sum()/df2['loan_amnt'].sum())

In [16]:
catboost_selected = combined.query('prediction==1')
catboost_selected['PnL'].sum()/catboost_selected['loan_amnt'].sum()

0.12441867617314067

In [22]:
purp_filter,ROI = calculate_ROI(catboost_selected, 'purpose',['small_business'])
print(ROI)

0.12445261685008002


In [25]:
emplen_filter, ROI = calculate_ROI(catboost_selected, 'emp_length', [-1])
print(ROI)

0.1246062141859198


In [26]:
homeown_filter, ROI = calculate_ROI(catboost_selected, 'home_ownership', ['RENT'])
print(ROI)

0.12570817213444574


In [27]:
catboost_selected.home_ownership.value_counts()

MORTGAGE    231931
RENT        164485
OWN          43062
OTHER          109
NONE            31
ANY              2
Name: home_ownership, dtype: int64

In [28]:
combo_filter, ROI = calculate_ROI(emplen_filter, 'home_ownership',['RENT'])
print(ROI)

0.12595773311079145


In [36]:
state_filter, ROI = calculate_ROI(combo_filter, 'addr_state', ['NV'])
print(ROI)

0.12607793368048906


In [37]:
inc_filter = state_filter.query('annual_inc>=42000')
inc_filter['PnL'].sum()/inc_filter['loan_amnt'].sum()

0.12638427622307724

In [40]:
purp_filter,ROI = calculate_ROI(inc_filter, 'purpose',['small_business'])
print(ROI)

0.1264885713553253
